## NER (Named Entity Recognition) of Banking77 dataset

In [78]:
%matplotlib inline
import re
import nltk
from datasets import load_dataset
from ipywidgets import IntProgress
import pandas as pd
import spacy
from spacy import displacy
import numpy as np
!python3 -m spacy download en_core_web_sm

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
     |████████████████████████████████| 13.9 MB 15.6 MB/s            
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
# Sample Data 
data = load_dataset("banking77",split="train").to_pandas()

query_column = "text"   # Column with the queries
label_column = "label"  # Column with intent names

########################
# ADD CAZOO DATA HERE
# load in the csv (or via any other format) into a pandas dataframe
# Set the "query_column" variable to the name of the column with the query text
# Set the "label_column" variable to the name of the column with intent names
# Then comment out the lines above
########################
#
# data = pd.read_csv("")
#
# query_column = "text"   # Column with the queries
# label_column = "label"  # Column with intent names
#

data.head()

NameError: name 'load_dataset' is not defined

In [12]:
ner = spacy.load("en_core_web_sm")

In [83]:
text = ner(data["text"][2])
len(text.ents)
displacy.render(text,style="ent",jupyter=True)

In [79]:
def apply_ner(text):
    text = text.lower()
    ner_text = ner(text)
    ner_tokens = [(word.text,word.label_) for word in ner_text.ents]
    if ner_tokens == []:
        return np.nan
    return ner_tokens
    
print(apply_ner(data["text"][1]))

[('2 weeks', 'DATE')]


In [80]:
data["ner"] = data["text"].apply(apply_ner)

In [102]:
data[:20]

,text,label,ner,ner_token_len
0,I am still waiting on my card?,11,NaN,0
1,What can I do if my card still hasn't arrived ...,11,"[(2 weeks, DATE)]",1
2,I have been waiting over a week. Is the card s...,11,"[(a week, DATE)]",1
3,Can I track my card while it is in the process...,11,NaN,0
4,"How do I know if I will get my card, or if it ...",11,NaN,0
5,When did you send me my new card?,11,NaN,0
6,Do you have info about the card on delivery?,11,NaN,0
7,What do I do if I still have not received my n...,11,NaN,0
8,Does the package with my card have tracking?,11,NaN,0
9,I ordered my card but it still isn't here,11,NaN,0


In [92]:
type(data["ner"][1])

list

In [93]:
def count_tokens(ner_tokens):
    if ner_tokens == np.nan or type(ner_tokens) == float:
        return 0
    ner_tokens = list(ner_tokens)
    return len(ner_tokens)

In [94]:
data["ner_token_len"] = data["ner"].apply(count_tokens)

In [98]:
data["ner_token_len"].value_counts()

0    8833
1    1004
2     132
3      25
4       5
5       3
6       1
Name: ner_token_len, dtype: int64

In [101]:
data[["ner","ner_token_len"]].sort_values("ner_token_len",ascending=False)

,ner,ner_token_len
6991,"[(belgium, GPE), (luxembourg, GPE), (austria, ...",6
3905,"[(10 minutes, TIME), (one hour, TIME), (18 yea...",5
5997,"[(up to one day, DATE), (one, CARDINAL), (sepa...",5
6431,"[(earlier today, TIME), (first, ORDINAL), (sec...",5
3934,"[(10 minutes, TIME), (18 years of age, DATE), ...",4
...,...,...
3584,NaN,0
3585,NaN,0
3586,NaN,0
3587,NaN,0


In [100]:
data["ner"][3905]

[('10 minutes', 'TIME'),
 ('one hour', 'TIME'),
 ('18 years of age or', 'DATE'),
 ('switzerland', 'GPE'),
 ('european', 'NORP')]